In [8]:
using CSV, DataFrames
using FileIO, Dates
using DelimitedFiles

include("../region_cube_split/cubes.jl")
include("./motifs_analysis.jl")

volume_tetrahedrons (generic function with 1 method)

In [2]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:];

vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :];

In [3]:
vrancea, vrancea_cubes = region_cube_split(vrancea,side=5,energyRelease=true);

In [11]:
motif = "Triangle"
# motif = "Tetrahedron"

"Triangle"

In [12]:
filename= "./motifs" * motif * "_vrancea_5km.csv"
motifs = readdlm(filename, ',', Int64);
motifs;

In [13]:
motif_energy = total_mean_energy(motifs, vrancea, vrancea_cubes)

Dict{Any, Any} with 2312 entries:
  1144 => [1.06164e12, 4.3573e10]
  2108 => [3.06265e12, 1.03199e11]
  1175 => [3.11695e12, 1.12903e11]
  1953 => [2.19417e12, 8.58461e10]
  719  => [4.99602e11, 4.36058e10]
  1546 => [1.3779e12, 6.83807e10]
  1703 => [6.48054e12, 1.74651e11]
  1956 => [2.49914e12, 1.37161e11]
  2261 => [3.07928e12, 9.16835e10]
  2288 => [6.98603e12, 1.94934e11]
  1028 => [9.37478e11, 5.65458e10]
  699  => [2.09455e12, 6.99485e10]
  831  => [1.03681e12, 3.69834e10]
  1299 => [1.29191e12, 8.8249e10]
  1438 => [1.2565e12, 8.09358e10]
  1074 => [1.58424e12, 5.54168e10]
  319  => [4.62493e12, 1.27889e11]
  687  => [1.14613e12, 1.0504e11]
  1812 => [3.50642e12, 1.46098e11]
  ⋮    => ⋮

In [15]:
areas = area_triangles(motifs, vrancea_cubes)

Dict{Any, Any} with 2312 entries:
  1144 => 12.6804
  2108 => 37.2354
  1175 => 74.5312
  1953 => 12.4318
  719  => 74.3585
  1546 => 99.6018
  1703 => 0.0706402
  1956 => 12.5215
  2261 => 12.2484
  2288 => 37.2352
  1028 => 74.536
  699  => 0.0515364
  831  => 24.8242
  1299 => 61.8098
  1438 => 12.5353
  1074 => 12.4124
  319  => 12.4519
  687  => 49.6237
  1812 => 0.0125032
  ⋮    => ⋮

In [16]:
# motif = "Triangle"
motif = "Tetrahedron"

"Tetrahedron"

In [17]:
filename= "./motifs" * motif * "_vrancea_5km.csv"
motifs = readdlm(filename, ',', Int64);
motifs;

In [18]:
motifs

21977×4 Matrix{Int64}:
  652  577  189    2
  652  577  192    2
 1281  353  168    2
 1281  353  336    2
  480   56    4    3
 1390  169   42   22
   81   82   42   24
   81   82   76   24
   78   79   36   26
  933  917  163   27
    ⋮            
  181  666  151   15
  181  376  160   56
  181  330  329  242
  181  919  189   46
  181  919  566  141
  181  919  131  141
  181  919  131   56
  181  919  131   15
  181  919  131   66

In [19]:
motif_energy = total_mean_energy(motifs, vrancea, vrancea_cubes)

Dict{Any, Any} with 21977 entries:
  11950 => [1.54193e12, 7.31641e10]
  1703  => [4.8157e11, 2.06604e10]
  12427 => [1.63154e12, 5.0651e10]
  7685  => [9.99835e12, 4.06294e11]
  18374 => [5.59464e12, 1.61607e11]
  3406  => [2.09461e13, 1.38262e12]
  1090  => [2.45345e12, 9.88382e10]
  2015  => [5.7741e13, 1.96905e12]
  18139 => [2.48052e12, 1.06015e11]
  17088 => [5.40481e12, 1.65503e11]
  11280 => [2.69321e13, 1.53375e12]
  16805 => [2.61405e12, 1.0413e12]
  3220  => [2.51717e12, 1.00208e11]
  11251 => [2.19959e13, 1.43391e12]
  422   => [1.7971e13, 6.00658e11]
  15370 => [2.91777e15, 1.38809e14]
  15859 => [1.59227e12, 8.40716e10]
  4030  => [2.18481e12, 1.58372e11]
  8060  => [8.41971e11, 3.78217e10]
  ⋮     => ⋮

In [20]:
voluems = volume_tetrahedrons(motifs,vrancea_cubes)

Dict{Any, Any} with 21977 entries:
  11950 => 1.023
  1703  => 1.07235
  12427 => 1.54003
  7685  => 0.225364
  18374 => 0.159408
  3406  => 0.247584
  1090  => 0.473368
  2015  => 0.54375
  18139 => 1.53502
  17088 => 0.174772
  11280 => 0.139509
  16805 => 1.13957
  3220  => 0.661264
  11251 => 0.614917
  422   => 0.0545653
  15370 => 0.0909343
  15859 => 0.255473
  4030  => 0.231534
  8060  => 0.156393
  ⋮     => ⋮